In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos, downselect
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box
from statmechlib.read_write import params_to_pickle

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
#stats_file = 'stats_manyall' # 'stats_samples'
#target_file = 'target_manyall' # 'target_samples'
stats_file = 'stats_all_samples' # 'stats_samples'
target_file = 'target_all_samples' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [6]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)

with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [7]:
# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
# edens_knots = [ 2.22, 2.5,  2.6946925, 2.8663175, 3.1,  3.5,  4.5]

# pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525, 5.401695, 5.4604375]

# pair_knots = [2.10, 2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525,5.401695,5.4604375, 5.54,5.58,5.62]             
#              5.18, 5.38, 5.58, 5.62]


#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.5]
#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.62]
#edens_knots = [2.22, 2.5, 2.8663175000000001, 3.1, 3.5, 4.62]

pair_knots = [2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
              3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
              4.8953, 5.089755, 5.3429525,5.401695,5.4604375]  
edens_knots = [2.5, 3.1, 3.5, 4.9]



In [8]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004]}
{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004]}


In [9]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [10]:
# for key in sorted(targets.keys()):
#     for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
#         n = len(xyz)
#         vol = np.linalg.det(box)
#         corr = float(n*n)/vol
#         ene.append([corr])

In [11]:
for i, box in enumerate(targets['eos']['box']):
    print(i, box[0,0])

(0, 6.3724472324999999)
(1, 7.0096919557500001)
(2, 7.6469366790000013)
(3, 8.2841814022500007)
(4, 8.9214261255)
(5, 9.5586708487499994)
(6, 10.195915572000001)
(7, 10.833160295250002)
(8, 11.470405018499999)
(9, 12.107649741749999)
(10, 12.744894465)
(11, 13.382139188250001)
(12, 14.0193839115)
(13, 14.656628634749998)
(14, 15.293873357999999)
(15, 15.93111808125)
(16, 16.568362804499998)
(17, 17.205607527750001)
(18, 17.842852251)
(19, 18.480096974249999)
(20, 19.117341697499999)


In [12]:
8.9214261255/12.74489446*2.74

1.9180000007524585

In [13]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [14]:
targets['eos']['box'] = targets['eos']['box'][6:9]
targets['eos']['xyz'] = targets['eos']['xyz'][6:9]
targets['eos']['temp'] = targets['eos']['temp'][6:9]
targets['eos']['beta'] = targets['eos']['beta'][6:9]
targets['eos']['energy'] = targets['eos']['energy'][6:9]
targets['eos']['forces'] = targets['eos']['forces'][6:9]
stats['eos']['energy'] = stats['eos']['energy'][6:9]
stats['eos']['forces'] = stats['eos']['forces'][6:9]

In [15]:
targets['relax']['box'][1] = targets['inf']['box'][0]
targets['relax']['xyz'][1] = targets['inf']['xyz'][0]
targets['relax']['temp'][1] = targets['inf']['temp'][0]
targets['relax']['beta'][1] = targets['inf']['beta'][0]
targets['relax']['energy'][1] = targets['inf']['energy'][0]
targets['relax']['forces'][1] = targets['inf']['forces'][0]
stats['relax']['energy'][1] = stats['inf']['energy'][0]
stats['relax']['forces'][1] = stats['inf']['forces'][0]

In [16]:
targets['relax']['energy']

[-1139.2,
 -2.9688679595213743e-239,
 -1136.4010143821092,
 -1136.7508187321096,
 -1190.2611772847656,
 -1127.1746971578905]

In [23]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.01
targets['eos_bcc']['weight'] = 0.0
targets['relax']['weight'] = 0.1
targets['inf']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [24]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('eos', 0.01),
 ('vac_npt_langevin_2000K', 1.0),
 ('md', 1.0),
 ('inf', 0.0),
 ('relax', 0.1),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('eos_bcc', 0.0),
 ('liq_5000K', 1.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('vacint', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('bain', 0.0)]

In [25]:
targets['eos']['temp'] = [5000.0, 5000., 5000., 5000.]
targets['eos']['beta'] = [1/5000.0, 1/5000., 1/5000., 1/5000.]

In [40]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])

# start_params.append([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
#          3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
#         -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
#         -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
#          7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
#         -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
#          1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
#          2.18327852e+00,   6.63152141e-02, 0.0])

# start_params.append([ -2.95016477e+00,   3.78177969e-04,  -5.21095611e+02,
#          3.09845244e+02,  -4.69222440e+02,   3.95186037e+02,
#         -1.11630659e+02,   5.36915127e+00,   1.52984702e+01,
#         -1.54915170e+01,   7.53107481e+00,  -2.98591742e+00,
#          7.55346307e-01,  -1.00678488e+00,   1.96634582e+00,
#         -8.71560927e-01,  -1.41820832e-01,  -1.46508000e+00,
#          1.42936365e+00,   7.23805795e+01,   0.0, 9.18071460e-01,
#          2.04259992e+00,   6.28575567e-02,   7.62804667e-03])

# start_params.append([ -3.27882058e+00,  -9.77681860e-04,  -5.95486537e+02,
#          3.42401535e+02,  -4.15497479e+02,   3.71316746e+02,
#         -1.07274610e+02,  -2.93282898e+00,   1.51298572e+01,
#         -1.37454674e+01,   8.16724633e+00,  -3.16850352e+00,
#          8.74607042e-01,  -1.07878768e+00,   1.97939820e+00,
#         -8.26149646e-01,  -1.62437297e-01,  -1.45694910e+00,
#          1.42153527e+00,   1.29203135e+02,  -9.25284676e+00,
#          2.63967417e+00,   2.06410900e+00,   6.03258240e-02*2])

# start_params.append([ -3.56709315e+00,  -2.97358686e-03,  -4.37632666e+02,
#          3.35095381e+02,  -2.95083648e+02,   2.94263473e+02,
#         -8.57731610e+01,  -1.62036228e+01,   1.30838141e+01,
#         -6.46551376e+00,   8.01826587e+00,  -3.58786095e+00,
#          1.25735987e+00,  -1.21627097e+00,   2.04033545e+00,
#         -8.04702913e-01,  -1.92666480e-01,  -1.44083371e+00,
#          1.41816098e+00,   1.70360405e+02,  -1.89070431e+01,
#          7.47139804e+00,   1.20540194e+00,   1.34588181e-01, 0.0])

# start_params.append([ -3.89054208e+00,  -2.99469508e-03,  -7.60325091e+02,
#          2.25922421e+02,   4.31682139e+01,  -2.00879587e+01,
#          3.15977299e+01,  -1.41216383e+01,   1.40729549e+01,
#         -2.29741069e-01,   5.66020784e-01,   9.50813370e-01,
#         -7.07877506e-01,  -6.33885976e-01,   2.11514130e+00,
#         -1.07084535e+00,  -1.99471358e-01,  -1.53976066e+00,
#          1.54749579e+00,   1.94668273e+02,  -1.93218649e+00,
#          4.87330211e+00,  -4.95019886e-01,   1.54561460e-01])

# start_params.append([ -3.01252922e+00,   3.34774026e-03,  -3.63245948e+02,
#          7.08473344e+01,   5.18269140e+00,  -4.21852665e+01,
#          5.03986439e+01,  -3.49299323e+01,   9.39050841e+00,
#          7.10916965e+00,   2.53141800e+00,  -1.27714028e-01,
#         -8.13152443e-01,  -5.45294030e-01,   2.19662624e+00,
#         -1.03021233e+00,  -2.37376783e-01,  -1.54945600e+00,
#          1.51410989e+00,   8.93873263e+01,  -2.44415218e+01,
#          1.44237601e+01,  -2.48517486e-01,   1.57735701e-01,
#         0.0])

# start_params.append([ -2.76762846e+00,   1.89330096e-03,  -3.78969507e+02,
#          4.97375946e+01,   5.61080362e+01,  -8.13991417e+01,
#          6.18213043e+01,  -3.36881958e+01,   9.64679581e+00,
#          6.37021438e+00,   2.56205745e+00,  -6.89271288e-02,
#         -8.28256696e-01,  -5.90663693e-01,   2.19738636e+00,
#         -9.78519868e-01,  -2.34054246e-01,  -1.56404102e+00,
#          1.50934562e+00,   1.25199350e+02,  -2.58933860e+01,
#          1.60206690e+01,  -2.46344691e-01,   1.98323959e-01,
#         0.0])

# start_params.append([ -2.96336623e+00,   9.35841772e-04,  -4.69405427e+02,
#          6.56489326e+01,   6.59098372e+01,  -6.95558862e+01,
#          5.05321941e+01,  -2.68901225e+01,   6.12983979e+00,
#          8.40257564e+00,   2.03599185e+00,   1.44655457e-01,
#         -8.07626370e-01,  -6.77900254e-01,   2.19493850e+00,
#         -9.02854774e-01,  -2.35346587e-01,  -1.56947636e+00,
#          1.49891405e+00,   1.54692423e+02,  -1.84128239e+01,
#          1.40416342e+01,  -4.74336808e-01,   2.07514499e-01])

# start_params.append([ -2.83157185e+00,   8.39115358e-04,  -4.21330968e+02,
#          5.62354559e+01,   6.55868693e+01,  -7.39097694e+01,
#          5.32753168e+01,  -2.90768080e+01,   9.73480693e+00,
#          5.86938247e+00,   2.40344261e+00,   1.33807130e-01,
#         -7.34188649e-01,  -8.53691403e-01,   2.20105517e+00,
#         -7.61586871e-01,  -2.42532670e-01,  -1.58457873e+00,
#          1.47655687e+00,   0.0, 1.48755630e+02,  0.0, -1.98109911e+01,
#          1.44803287e+01,  -1.17489342e-01,   2.06651900e-01])

# start_params.append([ -3.91274686e+00,  -5.02814282e-03,  -3.60607144e+02,
#          3.08180837e+02,  -2.45731624e+02,   2.57711860e+02,
#         -7.57560376e+01,  -3.26396342e+01,   1.25979333e+01,
#         -3.17982854e+00,   7.88108320e+00,  -3.46868286e+00,
#          1.29424331e+00,  -1.31629778e+00,   2.05422384e+00,
#         -8.33747039e-01,  -1.75108086e-01,  -1.43404985e+00,
#          1.45403961e+00,   1.43868098e+02,  -2.72551797e+01,
#          8.05592823e+00,   1.03649030e+00,   1.04857599e-01,
#         -4.20237149e+01])

# start_params.append([ -3.34519702e+00,   7.66822271e-04,  -4.03553412e+02,
#          1.55282185e+01,   2.90516401e+01,  -6.23087513e+01,
#          8.48184009e+01, -3.48806112e+01,   9.13550274e+00,
#          5.96096071e+00,   2.54062374e+00,   2.39873080e-01,
#         -7.19239652e-01,  -1.04267291e+00,   2.29366373e+00,
#         -7.05206149e-01,  -2.72279799e-01,  -1.55954882e+00,
#          1.45964251e+00,  -2.01431222e+02,   3.24297105e+01,
#          3.81664135e+01,  -2.18948938e+01,   1.02484895e+01,
#          2.20300623e-01,   1.13874983e-01])

# start_params.append([ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   0.0, 1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01])

# start_params.append([
#         -4.20657287, -0.00479083415,
#         -4.25919648e+03,   1.39193989e+02,   2.95719230e+01,
#         -1.18147968e+01,   1.70731105e+01,   4.70939187e+01,
#          8.92934163e+00,  -8.86791838e+00,  -1.76587802e+00,
#          1.76893096e+00,  -1.34741230e+00,  -4.32516046e-01,
#          1.42269060e+00,  -4.94492898e-01,  -6.03661025e-02,
#         -5.84036559e-01,   5.77793165e-01,
#         -3037.16289, 125.775067, 32.7634816,
#         -1.80274854, -1.77552231, 0.210521327
# ])

# start_params.append([-4.22162443, -0.00407859333,
#                      3478.84405,113.46408,-0.00903949895,1.35993297,
#                      12.8147257,46.4943117,6.83202558,-7.30628605,
#                      -1.99295103,1.90244807,-1.31365252,-0.804929717,
#                      1.98644116,-0.488771708,-0.0443359448,-1.46775185,
#                      1.19532238,-0.00538433431,-0.00128544755,-0.0025275236,
#                      2965.39266, 95.3494422, 31.2833793, -1.50835193, -1.76957075,0.205860236
#                     ]
# )

# start_params.append([ -4.19976555e+00,  -3.91095024e-03,   4.11762114e+03,
#          1.06506552e+02,   5.29253551e-01,   1.24324874e+00,
#          1.19653127e+01,   4.68002406e+01,   6.67699247e+00,
#         -7.04927872e+00,  -2.05647856e+00,   1.95042373e+00,
#         -1.33520940e+00,  -8.11118459e-01,   1.99354705e+00,
#         -4.84402488e-01,  -4.71892464e-02,  -1.46523753e+00,
#          1.19717936e+00,  -6.80779121e-03,  -3.49337274e-03,
#         -3.76805537e-03,   3.51929579e+03,   9.27623682e+01,
#          3.15808969e+01,  -1.44019460e+00,  -1.78665037e+00,
#          2.05532639e-01])

# start_params.append([ -4.16353110e+00,  -3.64115525e-03,   0.0, 4.52561445e+03,
#          1.07488634e+02,  -1.52604631e+01,   8.88626493e+00,
#          1.07120352e+01,   4.61310765e+01,   6.70641338e+00,
#         -7.14000617e+00,  -2.02174483e+00,   1.98976760e+00,
#         -1.35491449e+00,  -8.12592118e-01,   1.99278451e+00,
#         -4.84145757e-01,  -4.78673818e-02,  -1.46201396e+00,
#          1.19810233e+00,  -7.36954413e-03,  -3.88205423e-03,
#         -5.12566656e-03,   3.98736133e+03,   8.58053818e+01,
#          3.17983469e+01,  -1.52861585e+00,  -1.77508182e+00,
#          2.07405945e-01])

# start_params.append([ -4.08460829e+00,  -1.38588599e-03,  -1.32507730e+04,
#          1.93928425e+03,   2.05970852e+01,  -2.37669887e+02,
#          1.21943656e+02,  -3.24296128e+01,   6.26409270e+01,
#          8.36697799e+00,  -9.54708966e+00,  -2.06899378e+00,
#          1.75611005e+00,  -1.35952781e+00,  -7.35465370e-01,
#          1.98598708e+00,  -4.73450090e-01,  -4.99940237e-02,
#         -1.45135718e+00,   1.19557147e+00,   1.75773867e-04,
#         -1.84569288e-02,  -2.63221509e-02,   1.65754590e+03,
#         -1.03315521e+02,   3.86317998e+01,  -1.37580192e+00,
#         -2.15109594e+00,   2.00163255e-01])

# start_params.append([-5.946454, -0.049477, 
#                      9.60851701e+01, -1.84410924e+02,   9.35784080e+01,  -7.98358265e+00,
#                    7.47034093e+00,  -1.52756044e+00,   1.25205933e+00,
#                    1.63082162e+00,  -1.41854775e+00,  -8.19936046e-01,
#                    1.98013514e+00,  -6.96430180e-01,   3.04546910e-02,
#                   -1.63131143e+00,   1.38409896e+00,
#                      -4.20429108,  0.5182177 ,  0.05627208,  0.03441642])

start_params.append([ -5.94255002e+00,  -4.52753307e-03,  -8.98973159e+01,
         1.58307190e+02,  -7.12304117e+01,   6.46096124e+00,
         3.42388839e-01,   6.02193303e+00,   1.71026053e+00,
         8.61616548e-01,  -1.24021132e+00,  -7.52778245e-01,
         2.16478392e+00,  -6.80500465e-01,   8.35831474e-03,
        -1.70858135e+00,   1.33828368e+00,   1.06188317e+01,
         2.26603603e+00,  -1.52122221e-02,   2.00566458e-02])

multi_pars = start_params
print(multi_pars)

[[-5.94255002, -0.00452753307, -89.8973159, 158.30719, -71.2304117, 6.46096124, 0.342388839, 6.02193303, 1.71026053, 0.861616548, -1.24021132, -0.752778245, 2.16478392, -0.680500465, 0.00835831474, -1.70858135, 1.33828368, 10.6188317, 2.26603603, -0.0152122221, 0.0200566458]]


In [41]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [42]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-5.94255002, -0.00452753307, -89.8973159, 158.30719, -71.2304117, 6.46096124, 0.342388839, 6.02193303, 1.71026053, 0.861616548, -1.24021132, -0.752778245, 2.16478392, -0.680500465, 0.00835831474, -1.70858135, 1.33828368, 10.6188317, 2.26603603, -0.0152122221, 0.0200566458])
loss 3.10077242835e-06 3.10077242835e-06 0.0
('Best params:', [(3.1007724283473991e-06, [-5.94255002, -0.00452753307, -89.8973159, 158.30719, -71.2304117, 6.46096124, 0.342388839, 6.02193303, 1.71026053, 0.861616548, -1.24021132, -0.752778245, 2.16478392, -0.680500465, 0.00835831474, -1.70858135, 1.33828368, 10.6188317, 2.26603603, -0.0152122221, 0.0200566458])])
('ieam:', 0)
loss 3.10077242835e-06 3.10077242835e-06 0.0
loss 0.00660817930484 0.00660817930484 0.0
loss 3.19412507666e-06 3.19412507666e-06 0.0
loss 3.88869277294e-06 3.88869277294e-06 0.0
loss 1.23697311363e-05 1.23697311363e-05 0.0
loss 9.55938315448e-06 9.55938315448e-06 0.0
loss 3.93770644263e-06 3.93770644263e-06 0.0
loss 3.11938

loss 8.98401385405e-06 8.98401385405e-06 0.0
loss 4.10151395287e-06 4.10151395287e-06 0.0
loss 8.05693860099e-06 8.05693860099e-06 0.0
loss 3.80374250679e-06 3.80374250679e-06 0.0
loss 8.41761408573e-06 8.41761408573e-06 0.0
loss 3.7530287676e-06 3.7530287676e-06 0.0
loss 5.62873743338e-06 5.62873743338e-06 0.0
loss 6.84001153961e-06 6.84001153961e-06 0.0
loss 3.73929501199e-06 3.73929501199e-06 0.0
loss 5.69482532528e-06 5.69482532528e-06 0.0
loss 3.7455346703e-06 3.7455346703e-06 0.0
loss 6.3879260408e-06 6.3879260408e-06 0.0
loss 3.64781391767e-06 3.64781391767e-06 0.0
loss 5.3247948644e-06 5.3247948644e-06 0.0
loss 3.65028693686e-06 3.65028693686e-06 0.0
loss 5.80931171602e-06 5.80931171602e-06 0.0
loss 3.57644148328e-06 3.57644148328e-06 0.0
loss 5.29642670676e-06 5.29642670676e-06 0.0
loss 3.47482707875e-06 3.47482707875e-06 0.0
loss 5.36067921983e-06 5.36067921983e-06 0.0
loss 3.43566033375e-06 3.43566033375e-06 0.0
loss 5.53010699833e-06 5.53010699833e-06 0.0
loss 3.40526833692

loss 3.10210521395e-06 3.10210521395e-06 0.0
loss 3.10618072775e-06 3.10618072775e-06 0.0
loss 3.10216330893e-06 3.10216330893e-06 0.0
loss 3.10516159357e-06 3.10516159357e-06 0.0
loss 3.10195390116e-06 3.10195390116e-06 0.0
loss 3.10334456737e-06 3.10334456737e-06 0.0
loss 3.104163445e-06 3.104163445e-06 0.0
loss 3.10194178505e-06 3.10194178505e-06 0.0
loss 3.10491152548e-06 3.10491152548e-06 0.0
loss 3.10180631229e-06 3.10180631229e-06 0.0
loss 3.10516194655e-06 3.10516194655e-06 0.0
loss 3.10168048198e-06 3.10168048198e-06 0.0
loss 3.10457112824e-06 3.10457112824e-06 0.0
loss 3.10175170144e-06 3.10175170144e-06 0.0
loss 3.10440319998e-06 3.10440319998e-06 0.0
loss 3.10171143855e-06 3.10171143855e-06 0.0
loss 3.10317093497e-06 3.10317093497e-06 0.0
loss 3.10349764015e-06 3.10349764015e-06 0.0
loss 3.10168415903e-06 3.10168415903e-06 0.0
loss 3.10354838342e-06 3.10354838342e-06 0.0
loss 3.10166518598e-06 3.10166518598e-06 0.0
loss 3.10377440523e-06 3.10377440523e-06 0.0
loss 3.1016004

loss 3.10080612562e-06 3.10080612562e-06 0.0
loss 3.10077073941e-06 3.10077073941e-06 0.0
loss 3.10079757958e-06 3.10079757958e-06 0.0
loss 3.10077153778e-06 3.10077153778e-06 0.0
loss 3.10079731131e-06 3.10079731131e-06 0.0
loss 3.10077113341e-06 3.10077113341e-06 0.0
loss 3.10079829604e-06 3.10079829604e-06 0.0
loss 3.10077020342e-06 3.10077020342e-06 0.0
loss 3.10078783775e-06 3.10078783775e-06 0.0
loss 3.10078748378e-06 3.10078748378e-06 0.0
loss 3.10079204687e-06 3.10079204687e-06 0.0
loss 3.10077009952e-06 3.10077009952e-06 0.0
loss 3.10079620282e-06 3.10079620282e-06 0.0
loss 3.10076944984e-06 3.10076944984e-06 0.0
loss 3.10078543601e-06 3.10078543601e-06 0.0
loss 3.10079362521e-06 3.10079362521e-06 0.0
loss 3.10076956971e-06 3.10076956971e-06 0.0
loss 3.10078543233e-06 3.10078543233e-06 0.0
loss 3.1007816351e-06 3.1007816351e-06 0.0
loss 3.10079374189e-06 3.10079374189e-06 0.0
loss 3.10076869202e-06 3.10076869202e-06 0.0
loss 3.10078566897e-06 3.10078566897e-06 0.0
loss 3.10076

loss 3.10075687666e-06 3.10075687666e-06 0.0
loss 3.10075541608e-06 3.10075541608e-06 0.0
loss 3.10075827331e-06 3.10075827331e-06 0.0
loss 3.10075497481e-06 3.10075497481e-06 0.0
loss 3.10075506357e-06 3.10075506357e-06 0.0
loss 3.10075770605e-06 3.10075770605e-06 0.0
loss 3.10075486839e-06 3.10075486839e-06 0.0
loss 3.10075399062e-06 3.10075399062e-06 0.0
loss 3.10075366799e-06 3.10075366799e-06 0.0
loss 3.10075251106e-06 3.10075251106e-06 0.0
loss 3.10075476031e-06 3.10075476031e-06 0.0
loss 3.10075379849e-06 3.10075379849e-06 0.0
loss 3.10075241748e-06 3.10075241748e-06 0.0
loss 3.10075057079e-06 3.10075057079e-06 0.0
loss 3.10075296639e-06 3.10075296639e-06 0.0
loss 3.10075470653e-06 3.10075470653e-06 0.0
loss 3.10075125625e-06 3.10075125625e-06 0.0
loss 3.10075679039e-06 3.10075679039e-06 0.0
loss 3.10075406496e-06 3.10075406496e-06 0.0
loss 3.10075474764e-06 3.10075474764e-06 0.0
loss 3.10075244961e-06 3.10075244961e-06 0.0
loss 3.10075035445e-06 3.10075035445e-06 0.0
loss 3.100

...

loss 3.02888775654e-06 3.02888775654e-06 0.0
loss 3.02888762087e-06 3.02888762087e-06 0.0
loss 3.02888775425e-06 3.02888775425e-06 0.0
loss 3.02888761901e-06 3.02888761901e-06 0.0
loss 3.02888776373e-06 3.02888776373e-06 0.0
loss 3.02888761853e-06 3.02888761853e-06 0.0
loss 3.02888766792e-06 3.02888766792e-06 0.0
loss 3.02888766798e-06 3.02888766798e-06 0.0
loss 3.02888770672e-06 3.02888770672e-06 0.0
loss 3.02888761237e-06 3.02888761237e-06 0.0
loss 3.02888765133e-06 3.02888765133e-06 0.0
loss 3.0288876603e-06 3.0288876603e-06 0.0
loss 3.02888768472e-06 3.02888768472e-06 0.0
loss 3.02888761151e-06 3.02888761151e-06 0.0
loss 3.02888770399e-06 3.02888770399e-06 0.0
loss 3.02888760716e-06 3.02888760716e-06 0.0
loss 3.02888764667e-06 3.02888764667e-06 0.0
loss 3.02888767707e-06 3.02888767707e-06 0.0
loss 3.02888761036e-06 3.02888761036e-06 0.0
loss 3.02888762527e-06 3.02888762527e-06 0.0
loss 3.028887616e-06 3.028887616e-06 0.0
loss 3.02888763695e-06 3.02888763695e-06 0.0
loss 3.028887647

loss 3.02888757896e-06 3.02888757896e-06 0.0
loss 3.0288875763e-06 3.0288875763e-06 0.0
loss 3.02888757811e-06 3.02888757811e-06 0.0
loss 3.02888757739e-06 3.02888757739e-06 0.0
loss 3.02888757839e-06 3.02888757839e-06 0.0
loss 3.02888757536e-06 3.02888757536e-06 0.0
loss 3.02888757526e-06 3.02888757526e-06 0.0
loss 3.02888757744e-06 3.02888757744e-06 0.0
loss 3.02888757874e-06 3.02888757874e-06 0.0
loss 3.02888757628e-06 3.02888757628e-06 0.0
loss 3.02888757724e-06 3.02888757724e-06 0.0
loss 3.02888757792e-06 3.02888757792e-06 0.0
loss 3.02888757657e-06 3.02888757657e-06 0.0
loss 3.02888757593e-06 3.02888757593e-06 0.0
loss 3.02888757723e-06 3.02888757723e-06 0.0
loss 3.0288875756e-06 3.0288875756e-06 0.0
loss 3.02888757711e-06 3.02888757711e-06 0.0
loss 3.02888757514e-06 3.02888757514e-06 0.0
loss 3.02888757817e-06 3.02888757817e-06 0.0
loss 3.02888757584e-06 3.02888757584e-06 0.0
loss 3.02888757733e-06 3.02888757733e-06 0.0
loss 3.02888757613e-06 3.02888757613e-06 0.0
loss 3.0288875

Process PoolWorker-23:
Process PoolWorker-22:
Process PoolWorker-24:
Process PoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  

loss 3.02888757702e-06 3.02888757702e-06 0.0


KeyboardInterrupt: 

In [49]:
# params_output = [ -4.72546533e+00,  -6.97068121e-03,  -3.51290250e+02,
#          1.63175272e+02,   9.43598046e+00,  -3.25684355e+00,
#          2.30446047e+01,   4.06678582e+01,   7.56509137e+00,
#         -7.51663487e+00,  -1.59146529e+00,   1.14098894e+00,
#         -8.25274736e-01,  -8.09954101e-01,   2.09984210e+00,
#         -6.52067950e-01,  -8.56530962e-02,  -1.49919134e+00,
#          1.27499075e+00,   1.07226915e+02,   2.36918896e+01,
#         -1.29592031e+00,  -1.49967198e+00,   2.14664790e-01]
# params_output = [[ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01]]

# params_output = [[ -4.20657287e+00,  -4.79083415e-03,  -4.28662319e+03,
#          1.39452507e+02,   2.91473879e+01,  -1.08844243e+01,
#          1.59102238e+01,   4.86110415e+01,   7.30591870e+00,
#         -8.24225202e+00,  -1.76541035e+00,   1.69386336e+00,
#         -1.23281961e+00,  -7.91007042e-01,   1.98078659e+00,
#         -4.92223582e-01,  -4.64721217e-02,  -1.47436944e+00,
#          1.19244103e+00,  -3.03716289e+03,   1.25775067e+02,
#          3.27634816e+01,  -1.80274854e+00,  -1.77552231e+00,
#          2.10521327e-01]]

# params_output = [[ -4.18519141e+00,  -3.73537266e-03,   3.96171752e+03,
#          1.18316128e+02,  -8.46735170e+00,   1.49884675e+01,
#          5.43882551e+00,   4.19584851e+01,   1.05383775e+01,
#         -8.44501744e+00,  -1.76769900e+00,   2.25806651e+00,
#         -1.68693854e+00,  -4.53202950e-01,   1.47888419e+00,
#         -4.72027075e-01,  -7.36382597e-02,  -5.90831325e-01,
#          5.77070122e-01,   3.49058280e+03,   9.94638189e+01,
#          2.96556203e+01,  -7.14356623e-01,  -1.77478736e+00,
#          2.05018897e-01]] # s2 = 1.5313539642322428e-06

# params_output = [[ -4.19976555e+00,  -3.91095024e-03,   4.11762114e+03,
#          1.06506552e+02,   5.29253551e-01,   1.24324874e+00,
#          1.19653127e+01,   4.68002406e+01,   6.67699247e+00,
#         -7.04927872e+00,  -2.05647856e+00,   1.95042373e+00,
#         -1.33520940e+00,  -8.11118459e-01,   1.99354705e+00,
#         -4.84402488e-01,  -4.71892464e-02,  -1.46523753e+00,
#          1.19717936e+00,  -6.80779121e-03,  -3.49337274e-03,
#         -3.76805537e-03,   3.51929579e+03,   9.27623682e+01,
#          3.15808969e+01,  -1.44019460e+00,  -1.78665037e+00,
#          2.05532639e-01]]

# params_output = [[ -4.16353110e+00,  -3.64115525e-03,   4.52561445e+03,
#          1.07488634e+02,  -1.52604631e+01,   8.88626493e+00,
#          1.07120352e+01,   4.61310765e+01,   6.70641338e+00,
#         -7.14000617e+00,  -2.02174483e+00,   1.98976760e+00,
#         -1.35491449e+00,  -8.12592118e-01,   1.99278451e+00,
#         -4.84145757e-01,  -4.78673818e-02,  -1.46201396e+00,
#          1.19810233e+00,  -7.36954413e-03,  -3.88205423e-03,
#         -5.12566656e-03,   3.98736133e+03,   8.58053818e+01,
#          3.17983469e+01,  -1.52861585e+00,  -1.77508182e+00,
#          2.07405945e-01]]

# params_output = [[ -4.08460829e+00,  -1.38588599e-03,  -1.32507730e+04,
#          1.93928425e+03,   2.05970852e+01,  -2.37669887e+02,
#          1.21943656e+02,  -3.24296128e+01,   6.26409270e+01,
#          8.36697799e+00,  -9.54708966e+00,  -2.06899378e+00,
#          1.75611005e+00,  -1.35952781e+00,  -7.35465370e-01,
#          1.98598708e+00,  -4.73450090e-01,  -4.99940237e-02,
#         -1.45135718e+00,   1.19557147e+00,   1.75773867e-04,
#         -1.84569288e-02,  -2.63221509e-02,   1.65754590e+03,
#         -1.03315521e+02,   3.86317998e+01,  -1.37580192e+00,
#         -2.15109594e+00,   2.00163255e-01]]

# params_output = [[ -5.94255002e+00,  -4.52753307e-03,  -8.98973159e+01,
#          1.58307190e+02,  -7.12304117e+01,   6.46096124e+00,
#          3.42388839e-01,   6.02193303e+00,   1.71026053e+00,
#          8.61616548e-01,  -1.24021132e+00,  -7.52778245e-01,
#          2.16478392e+00,  -6.80500465e-01,   8.35831474e-03,
#         -1.70858135e+00,   1.33828368e+00,   1.06188317e+01,
#          2.26603603e+00,  -1.52122221e-02,   2.00566458e-02]]

# params_output.append([ -6.67485922e+00,  -1.02376651e-02,   1.91540112e+02,
#         -1.02318453e+02,   3.55386850e+01,  -5.52132877e+00,
#          9.33351478e+00,   9.88401954e-01,   1.50327402e+00,
#         -5.01846513e-01,  -1.06459763e+00,  -5.05974054e-01,
#          2.00937835e+00,  -6.02252894e-01,   8.10245260e-04,
#         -1.64969075e+00,   1.29187809e+00,   4.66926359e+01,
#          2.30114335e+00,  -7.42024973e-01,   4.07443544e-02])

# params_output.append([ -6.02193727e+00,  -3.85371079e-03,  -6.97726900e+01,
#          1.34506659e+02,  -5.71483113e+01,   1.76207446e+00,
#          7.47456613e+00,  -6.72890122e-01,   2.76103862e+00,
#          7.76284266e-01,  -1.27164334e+00,  -7.67755950e-01,
#          2.18615528e+00,  -6.83040984e-01,   2.38050659e-03,
#         -1.70140052e+00,   1.33115736e+00,   1.35592664e+01,
#          1.60373079e+00,   1.47525215e-01,   1.65277513e-02])

params_output.append([ -5.85430083e+00,  -5.22915244e-03,  -3.84070447e+01,
         1.18120884e+02,  -5.29405790e+01,   1.27051644e+00,
         8.33161715e+00,  -2.35387541e+00,   3.06151106e+00,
         8.07273686e-01,  -1.28106541e+00,  -7.86145231e-01,
         2.20500009e+00,  -6.83596584e-01,  -3.70671719e-03,
        -1.70638158e+00,   1.33597525e+00,   2.23939837e+01,
         1.33742607e+00,   2.58862865e-01,   1.55687124e-02])

print(params_output)

[[-6.02193727, -0.00385371079, -69.77269, 134.506659, -57.1483113, 1.76207446, 7.47456613, -0.672890122, 2.76103862, 0.776284266, -1.27164334, -0.76775595, 2.18615528, -0.683040984, 0.00238050659, -1.70140052, 1.33115736, 13.5592664, 1.60373079, 0.147525215, 0.0165277513], [-5.85430083, -0.00522915244, -38.4070447, 118.120884, -52.940579, 1.27051644, 8.33161715, -2.35387541, 3.06151106, 0.807273686, -1.28106541, -0.786145231, 2.20500009, -0.683596584, -0.00370671719, -1.70638158, 1.33597525, 22.3939837, 1.33742607, 0.258862865, 0.0155687124]]


In [50]:
# convert lists of optimal parameters to dictionaries
params_dicts = []
for params in params_output:
    params_dicts.append(to_param_dict(params, stats['hyperparams']))

In [51]:
params_dicts

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [13.5592664, 1.60373079, 0.147525215, 0.0165277513],
   'embed': [-6.02193727, -0.00385371079],
   'lrcorr': [],
   'pair': [-69.77269,
    134.506659,
    -57.1483113,
    1.76207446,
    7.47456613,
    -0.672890122,
    2.76103862,
    0.776284266,
    -1.27164334,
    -0.76775595,
    2.18615528,
    -0.683040984,
    0.00238050659,
    -1.70140052,
    1.33115736]}},
 {'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,


In [52]:
file_name = os.path.join(working, "params_out.pickle")

for params in params_dicts:
    params_to_pickle(params, file_name)

In [53]:
with open(os.path.join(working, 'params_out' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [54]:
retreived_params

[{'hyperparams': {u'edens': [2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.5],
   u'pair': [2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [106.853821,
    27.4234479,
    -1.26300179,
    -1.66378666,
    0.224848394],
   'embed': [-4.65804603, -0.00756457588],
   'lrcorr': [],
   'pair': [-168.77752,
    156.052349,
    14.2649908,
    -4.55941009,
    19.761334,
    48.4204936,
    6.4946677,
    -6.62233324,
    -2.16547494,
    1.30932417,
    -0.83328148,
    -0.832333597,
    2.09417158,
    -0.619566076,
    -0.0953112547,
    -1.49756931,
    1.2716794]}},
 {'hyperparams': {u'edens': [2.2200000000000002,

In [39]:
retreived_params[-2:]

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [10.6188317, 2.26603603, -0.0152122221, 0.0200566458],
   'embed': [-5.94255002, -0.00452753307],
   'lrcorr': [],
   'pair': [-89.8973159,
    158.30719,
    -71.2304117,
    6.46096124,
    0.342388839,
    6.02193303,
    1.71026053,
    0.861616548,
    -1.24021132,
    -0.752778245,
    2.16478392,
    -0.680500465,
    0.00835831474,
    -1.70858135,
    1.33828368]}},
 {'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.694692499999999